In [1]:
# Install required packages
#!pip install -U langchain-huggingface faiss-cpu sentence-transformers langchain-openai
!pip install -U langchain-community faiss-cpu sentence-transformers huggingface-hub langchain-core

import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.messages import SystemMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain.chains import LLMChain
from langchain_community.chat_models import ChatOpenAI

# Set your OpenAI API key (replace with your actual key)
os.environ['OPENAI_API_KEY'] = 'your_api_key_here'

# Initialize embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Sample product documentation - replace with your actual content
product_docs = {
    "product_a": [
        "Product A is our flagship enterprise solution with advanced features.",
        "To troubleshoot Product A, first check the system logs and verify network connectivity.",
        "Product A requires minimum 8GB RAM and 50GB disk space for optimal performance.",
        "Common error codes for Product A: 101 (connection timeout), 205 (authentication failure).",
        "Product A supports integration with all major CRM platforms through our API."
    ],
    "product_b": [
        "Product B is our cloud-based service offering real-time analytics.",
        "Product B subscription includes 24/7 support and monthly feature updates.",
        "To upgrade your Product B plan, visit the account settings page.",
        "Common Product B error codes: 301 (rate limit exceeded), 404 (resource not found).",
        "Product B mobile app is available on iOS and Android platforms."
    ],
    "product_c": [
        "Product C is our mobile application for field service technicians.",
        "Product C system requirements: iOS 14+ or Android 10+, 2GB RAM minimum.",
        "Product C sync settings can be configured in the app's preferences menu.",
        "For Product C offline mode, download necessary data before going to field sites.",
        "Product C integrates with both Product A and Product B for full functionality."
    ]
}

# Custom chunking strategy
tech_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    separators=["\n\n", "\n", ". ", "! ", "? ", " ", ""]
)

# Process documents and create vector stores
knowledge_bases = {}
for product, docs in product_docs.items():
    chunks = tech_splitter.create_documents(docs)
    knowledge_bases[product] = FAISS.from_documents(chunks, embeddings)

# Initialize memory with system message
memory = ConversationBufferWindowMemory(
    k=5,
    return_messages=True,
    memory_key="chat_history",
    system_message=SystemMessage(content=(
        "You are an enterprise customer support agent. "
        "Be professional, helpful, and concise. "
        "Base your answers on the provided product documentation. "
        "If you don't know an answer, offer to escalate the issue."
    ))
)

# Initialize LLM
llm = ChatOpenAI(temperature=0.2, model="gpt-3.5-turbo")

# Create prompt template
prompt_template = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a professional support agent."),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

# Create conversation chain
conversation_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    memory=memory,
    verbose=False
)

# Support tools implementation
def check_order_status(order_id: str) -> str:
    """Mock function to check order status"""
    return f"Order {order_id} status: Shipped (Expected delivery: May 15)"

def escalate_issue(description: str, urgency: str = "normal") -> str:
    """Mock function to escalate issues"""
    return f"Issue '{description}' has been escalated as {urgency} priority (Ticket: SR-{os.urandom(4).hex()})"

# Main query handler
def handle_query(user_query: str, product: str = None):
    try:
        # If product not specified, try to determine it
        if not product:
            query_lower = user_query.lower()
            if "product a" in query_lower or "producta" in query_lower:
                product = "product_a"
            elif "product b" in query_lower or "productb" in query_lower:
                product = "product_b"
            elif "product c" in query_lower or "productc" in query_lower:
                product = "product_c"

        # Check for special commands
        if "#orderstatus" in user_query:
            order_id = user_query.split("#orderstatus")[1].strip()
            return check_order_status(order_id)
        elif "#escalate" in user_query:
            parts = user_query.split("#escalate")[1].strip().split("|")
            desc = parts[0].strip()
            urgency = parts[1].strip() if len(parts) > 1 else "normal"
            return escalate_issue(desc, urgency)

        # Retrieve relevant context if product is known
        context = ""
        if product and product in knowledge_bases:
            docs = knowledge_bases[product].similarity_search(user_query, k=3)
            context = "\nRelevant information:\n" + "\n".join([doc.page_content for doc in docs])

        # Generate response
        response = conversation_chain.run(input=f"Customer query: {user_query}\n{context}")
        return response

    except Exception as e:
        return f"I encountered an error processing your request. Please try again or contact support. Error: {str(e)}"

# Test the system
if __name__ == "__main__":
    print("=== Enterprise Customer Support System ===")
    print("Try queries like:")
    print("- How do I troubleshoot Product A?")
    print("- What are the system requirements for Product C?")
    print("- #orderstatus ORD12345")
    print("- #escalate Can't login to Product B|urgent")
    print("Type 'exit' to quit\n")

    while True:
        query = input("Customer query: ")
        if query.lower() == 'exit':
            break
        response = handle_query(query)
        print("\nSupport Agent:", response, "\n")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
<ipython-input-1-81f0d08ea895>:60: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(
<ipython-input-1-81f0d08ea895>:73: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip in

=== Enterprise Customer Support System ===
Try queries like:
- How do I troubleshoot Product A?
- What are the system requirements for Product C?
- #orderstatus ORD12345
- #escalate Can't login to Product B|urgent
Type 'exit' to quit

Customer query: #escalate

Support Agent: Issue '' has been escalated as normal priority (Ticket: SR-5b6b84e3) 

Customer query: #orderstatus

Support Agent: Order  status: Shipped (Expected delivery: May 15) 

Customer query: exit
